# Dataset de Partidas de Dota 2
## Proyecto de minería de datos

### Integrantes :
* Tomas Núñez G.
* Sven Reisenegger M.
* Pablo Torres D

## Introducción
Dota es un juego del género "MOBA" (multiplayer online battle arena) y es uno de los videojuegos online mas populares de los últimos años, con peaks de 700 mil usuarios jugando simultáneamente. 

El juego consiste en partidas donde dos equipos de cinco jugadores cada uno se enfrentan para destruir la base enemiga. Al comienzo de una partida cada jugador elige un "héroe", que es el personaje que usará durante toda la partida. Todos los héroes tienen habilidades distintas, que los distinguen del resto. Actualmente se puede escoger de un rango de 115 héroes diferentes.

Dota posee además una escena competitiva bastante fuerte, donde durante todo el año se juegan torneos. El torneo más importante del año es el *International*, donde se reparten mas de 25 millones de dolares entre los 16 equipos participantes.



## Problema
En principio hay dos problemas que nos gustaría resolver. El primero consiste en hacer una predicción de los resultados de las partidas basándonos principalmente en los héroes que se escogieron al principio de esta. Esto se pretende realizar usando un sistema de redes neuronales entrenada con un subconjunto de los mismos datos. 

El otro problema tentativo a resolver consiste en establecer una correlación entre el uso del chat y las victorias/derrotas de los equipos. Para esto se observaría cuánto hablan en el chat los jugadores y qué tipo de lenguaje utilizan para luego ver si esto afecta las chances de ganar de sus equipos respectivos.

Para el segundo hito decidimos intentar resolver el problema, planteándonos como hipótesis que es posible predecir el 70% de los resultados de las partidas de dota 2 basándonos sólamente en qué héroe elige cada equipo.

## Datos
El dataset que tenemos consta de 16 tablas, de las cuales la mayoria contiene datos de distintos aspectos de cada partida y se relacionan mediante el ID de estas. Las tablas mas relevantes para nuestras hipotesis son:

1. *match_outcomes*: Contiene el equipo ganador de cada partida y las ids de cada jugador.
2. *players*: Contiene las ids de los heroes escogidos por cada jugador en cada partida.
3. *chat*: Contiene el registro del chat de cada partida, con cada mensaje formateado como un string
4. *player_time*: Muestra el oro y experiencia de cada jugador, entre otros, a lo largo de cada partida. Son un indicador de que tan bien va cada jugador a lo largo de la partida y podria agregarse al modelo predictor
5. *teamfights_players*: Muestra las peleas de equipo que se efectuan a lo largo de cada partida, y el desempeño de cada jugador en estas. Son un indicador de que tan bien va cada jugador a lo largo de la partida y podria agregarse al modelo predictor

## Pre-Procesamiento de los datos
Para procesar los datos requeridos para resolver el problema, se hizo un script en python que lee la columna *hero_id* de la tabla players y guarda un 1 en un vector de tamaño 113, en la posición *hero_id* si el héroe es escogido por el equipo 1 y un -1 si el héroe es escogido por el equipo 2.
Además de eso se guardaron en otro archivo los resultados de cada partida, indicando con un 1 si ganó el equipo 1 y con un -1 si ganó el equipo 2.
A continuación se presenta el script que se usó:


In [ ]:

players=open("players.csv","r")
players.readline()

match=open("match.csv","r")
match.readline()

x=np.ndarray([50000,113])
y=np.ndarray([50000])
for i in range(50000):
    for j in range(10):
        hero = int(players.readline().split(",")[2])
        if j<5:
            x[i, hero] = 1
        else:
            x[i, hero] = -1
    if match.readline().split(",")[9]=="True":
        y[i]=1
    else:
        y[i]=-1

## Experimentos
Luego de procesar los datos, estos quedaron en un formato ideal para ser entregados a los clasificadores de la librería scikit.learn.
El primer experimento realizado consistió en entrenar y probar una red neuronal de múltiples capas de perceptrones y analizar los resultados.
Luego de esto se probó el mismo experimento, pero en vez de eso se intentó con un clasificador de Support Vector Machine optimizado con SGD.

A continuación se muestran partes del código utilizado para realizar la clasificación.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

from sklearn.svm import SVC
import numpy as np

X = np.load('picks.npy')
y = np.load('winners.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=15, stratify = y)

#esta función entrena un clasificador y entrega un reporte para los datos.
def train_and_print(red, X_train, X_test, y_train, y_test):
    red.fit(X_train, y_train)
    y_pred = red.predict(X_test)
    print("Accuracy = ", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

Se probaron varias configuraciones de MLPClassifier y la que mejores resultado arrojó se muestra a continuación.
Como se puede ver, éste clasificador usa optimización SGD, función de activación de tangente hiperbólica y tiene tres capas ocultas de 60, 30 y 15 perceptrones, respectivamente.

In [10]:
redsgd6 = MLPClassifier(max_iter=500,solver='sgd', alpha=1e-5, hidden_layer_sizes=(60,30,15), 
                        random_state=1, activation= "tanh")
train_and_print(redsgd6, X_train, X_test, y_train, y_test)


Accuracy =  0.6062
             precision    recall  f1-score   support

       -1.0       0.59      0.57      0.58      4811
        1.0       0.62      0.64      0.63      5189

avg / total       0.61      0.61      0.61     10000



Cómo se ve en el reporte, el clasificador tiene una accuracy de 0.6062, que aún está a un 10% de diferencia de lo que se quiere alcanzar. En este caso accuracy es un buen indicador, ya que ambas clases están balanceadas.

## Conclusiones

Con los resultados obtenidos hasta el momento, no ha sido posible verificar la hipótesis. Esto se debe probablemente a que el resultado de una partida está definido por muchos mas factores que sólo quien elige a qué héroe. Otros factores que pueden afectar el resultado de una partida pueden incluir el nivel de habilidad de cada uno de los jugadores, su estado emocional, si la dinámica del equipo es buena, etc. Es por esto que se buscará agregar nuevas variables al clasificador, como por ejemplo agregar datos de los primeros minutos de la partida y se pueda llegar al objetivo de un 70% de precisión.
Dados los factores que afectan el problema, se considera que el resultado obtenidos (60% de accuracy) es bueno.

Otra opción es hacer que el clasificador usado entregue probabilidades de victoria en vez de resultados categóricos.